# SpaceX Falcon 9 First Stage Landing Prediction - SQL Analysis

## Project Overview

This notebook performs comprehensive SQL-based exploratory data analysis on SpaceX Falcon 9 launch data. The analysis leverages the dataset collected via the SpaceX API to extract insights about launch patterns, landing outcomes, and mission characteristics.

### Objectives

1. Execute SQL queries to understand launch site distributions
2. Analyze payload characteristics across different mission profiles
3. Investigate landing success patterns and trends
4. Examine booster reusability metrics
5. Generate insights for predictive modeling

### Dataset Schema

The analysis uses the dataset from `data_collection.ipynb` with the following structure:

- **FlightNumber**: Sequential launch identifier
- **Date**: Launch date
- **BoosterVersion**: Falcon 9 variant
- **PayloadMass**: Payload mass in kg
- **Orbit**: Target orbit designation
- **LaunchSite**: Launch facility
- **Outcome**: Landing success and type
- **Flights, GridFins, Reused, Legs**: Booster characteristics
- **LandingPad**: Landing location
- **Block, ReusedCount, Serial**: Booster metadata
- **Longitude, Latitude**: Geographic coordinates

## 1. Environment Setup

In [1]:
# Load SQL extension and import libraries
%load_ext sql
import sqlite3
import prettytable
import warnings

import pandas as pd

prettytable.DEFAULT = 'DEFAULT'

warnings.filterwarnings('ignore')

print("✓ Environment configured successfully")

✓ Environment configured successfully


## 2. Database Initialization

In [2]:
# Create SQLite database connection
con = sqlite3.connect("spacex_falcon9.db")
cur = con.cursor()

# Configure SQL magic to use this database
%sql sqlite:///spacex_falcon9.db

print("✓ Database connection established")

✓ Database connection established


In [3]:
# Load the dataset from data_collection.ipynb
df = pd.read_csv('spacex_falcon9_dataset.csv')

# Convert Date column to proper datetime format
df['Date'] = pd.to_datetime(df['Date'])

# Load data into SQLite database
df.to_sql('SPACEX_LAUNCHES', con, if_exists='replace', index=False, method='multi')

print(f"✓ Dataset loaded: {len(df)} records")
print(f"  Date range: {df['Date'].min()} to {df['Date'].max()}")

✓ Dataset loaded: 168 records
  Date range: 2010-06-04 00:00:00 to 2022-10-05 00:00:00


In [4]:
%%sql
-- Verify table structure
PRAGMA table_info(SPACEX_LAUNCHES);

 * sqlite:///spacex_falcon9.db
Done.


cid,name,type,notnull,dflt_value,pk
0,FlightNumber,INTEGER,0,None,0
1,Date,TIMESTAMP,0,None,0
2,BoosterVersion,TEXT,0,None,0
3,PayloadMass,REAL,0,None,0
4,Orbit,TEXT,0,None,0
5,LaunchSite,TEXT,0,None,0
6,Outcome,TEXT,0,None,0
7,Flights,INTEGER,0,None,0
8,GridFins,INTEGER,0,None,0
9,Reused,INTEGER,0,None,0


## 3. Exploratory SQL Queries

### 3.1 Launch Site Analysis

**Objective**: Identify unique launch sites and their usage patterns

In [5]:
%%sql
-- Display unique launch sites with launch counts
SELECT 
    LaunchSite,
    COUNT(*) AS LaunchCount,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM SPACEX_LAUNCHES), 2) AS Percentage
FROM SPACEX_LAUNCHES
GROUP BY LaunchSite
ORDER BY LaunchCount DESC;

 * sqlite:///spacex_falcon9.db
Done.


LaunchSite,LaunchCount,Percentage
CCSFS SLC 40,93,55.36
KSC LC 39A,49,29.17
VAFB SLC 4E,26,15.48


### 3.2 Launch Site Geographic Distribution

**Objective**: Examine launches from Cape Canaveral facilities

In [6]:
%%sql
-- List first 5 launches from CCSFS (Cape Canaveral)
SELECT 
    FlightNumber,
    Date,
    LaunchSite,
    BoosterVersion,
    Outcome
FROM SPACEX_LAUNCHES
WHERE LaunchSite LIKE 'CCSFS%'
ORDER BY Date ASC
LIMIT 5;

 * sqlite:///spacex_falcon9.db
Done.


FlightNumber,Date,LaunchSite,BoosterVersion,Outcome
1,2010-06-04 00:00:00,CCSFS SLC 40,Falcon 9,None None
2,2012-05-22 00:00:00,CCSFS SLC 40,Falcon 9,None None
3,2013-03-01 00:00:00,CCSFS SLC 40,Falcon 9,None None
5,2013-12-03 00:00:00,CCSFS SLC 40,Falcon 9,None None
6,2014-01-06 00:00:00,CCSFS SLC 40,Falcon 9,None None


### 3.3 Payload Mass Analysis by Orbit

**Objective**: Calculate total and average payload masses for different orbit types

In [7]:
%%sql
-- Analyze payload distribution across orbit types
SELECT 
    Orbit,
    COUNT(*) AS MissionCount,
    ROUND(SUM(PayloadMass), 2) AS TotalPayloadKg,
    ROUND(AVG(PayloadMass), 2) AS AvgPayloadKg,
    MIN(PayloadMass) AS MinPayloadKg,
    MAX(PayloadMass) AS MaxPayloadKg
FROM SPACEX_LAUNCHES
GROUP BY Orbit
ORDER BY TotalPayloadKg DESC
LIMIT 10;

 * sqlite:///spacex_falcon9.db
Done.


Orbit,MissionCount,TotalPayloadKg,AvgPayloadKg,MinPayloadKg,MaxPayloadKg
VLEO,54,776820.0,14385.56,13200.0,15600.0
GTO,31,156323.85,5042.7,3000.0,7076.0
ISS,32,153226.33,4788.32,677.0,12519.0
PO,13,111813.0,8601.0,500.0,15600.0
LEO,14,80492.55,5749.47,330.0,13260.0
SSO,11,47504.32,4318.57,475.0,8191.079109589042
MEO,5,19142.0,3828.4,3500.0,4400.0
GEO,2,12129.08,6064.54,3938.0,8191.079109589042
SO,1,8191.08,8191.08,8191.079109589042,8191.079109589042
None,1,8191.08,8191.08,8191.079109589042,8191.079109589042


### 3.4 Landing Outcome Success Rate

**Objective**: Analyze landing success rates by landing type

In [8]:
%%sql
-- Calculate landing success rates
SELECT 
    Outcome,
    COUNT(*) AS AttemptCount,
    ROUND(COUNT(*) * 100.0 / (SELECT COUNT(*) FROM SPACEX_LAUNCHES), 2) AS PercentageOfTotal
FROM SPACEX_LAUNCHES
GROUP BY Outcome
ORDER BY AttemptCount DESC;

 * sqlite:///spacex_falcon9.db
Done.


Outcome,AttemptCount,PercentageOfTotal
True ASDS,109,64.88
True RTLS,23,13.69
None None,19,11.31
False ASDS,7,4.17
True Ocean,5,2.98
None ASDS,2,1.19
False Ocean,2,1.19
False RTLS,1,0.6


### 3.5 First Successful Ground Landing

**Objective**: Identify the milestone of first successful RTLS (Return to Launch Site)

In [9]:
%%sql
-- Find first successful landing (any type)
SELECT 
    FlightNumber,
    Date,
    BoosterVersion,
    LaunchSite,
    Outcome,
    Serial
FROM SPACEX_LAUNCHES
WHERE Outcome LIKE 'True%'
ORDER BY Date ASC
LIMIT 1;

 * sqlite:///spacex_falcon9.db
Done.


FlightNumber,Date,BoosterVersion,LaunchSite,Outcome,Serial
7,2014-04-18 00:00:00,Falcon 9,CCSFS SLC 40,True Ocean,B1006


### 3.6 Booster Reusability Analysis

**Objective**: Identify boosters with successful drone ship landings and moderate payloads

In [10]:
%%sql
-- Successful ASDS landings with payload between 4000-6000 kg
SELECT 
    Serial AS BoosterSerial,
    BoosterVersion,
    Block,
    PayloadMass,
    Orbit,
    Date,
    ReusedCount
FROM SPACEX_LAUNCHES
WHERE Outcome = 'True ASDS'
  AND PayloadMass > 4000
  AND PayloadMass < 6000
ORDER BY Date ASC;

 * sqlite:///spacex_falcon9.db
Done.


BoosterSerial,BoosterVersion,Block,PayloadMass,Orbit,Date,ReusedCount
B1022,Falcon 9,2,4696.0,GTO,2016-05-06 00:00:00,0
B1026,Falcon 9,2,4600.0,GTO,2016-08-14 00:00:00,0
B1021,Falcon 9,2,5300.0,GTO,2017-03-30 00:00:00,1
B1031,Falcon 9,3,5200.0,GTO,2017-10-11 00:00:00,1
B1046,Falcon 9,5,5800.0,GTO,2018-08-07 00:00:00,3
B1059,Falcon 9,5,5000.0,ISS,2019-12-05 00:00:00,5


### 3.7 Temporal Landing Success Trends

**Objective**: Track improvement in landing success over time

In [11]:
%%sql
-- Landing outcomes by year
SELECT 
    STRFTIME('%Y', Date) AS Year,
    COUNT(*) AS TotalLaunches,
    SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) AS SuccessfulLandings,
    SUM(CASE WHEN Outcome LIKE 'False%' THEN 1 ELSE 0 END) AS FailedLandings,
    SUM(CASE WHEN Outcome = 'None None' THEN 1 ELSE 0 END) AS NoAttempt,
    ROUND(SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) * 100.0 / 
          NULLIF(SUM(CASE WHEN Outcome NOT LIKE 'None%' THEN 1 ELSE 0 END), 0), 2) AS SuccessRate
FROM SPACEX_LAUNCHES
GROUP BY Year
ORDER BY Year ASC;

 * sqlite:///spacex_falcon9.db
Done.


Year,TotalLaunches,SuccessfulLandings,FailedLandings,NoAttempt,SuccessRate
2010,1,0,0,1,None
2012,1,0,0,1,None
2013,3,0,1,2,0.0
2014,6,2,1,3,66.67
2015,6,2,2,1,50.0
2016,8,5,2,0,71.43
2017,18,15,0,3,100.0
2018,18,11,1,6,91.67
2019,10,9,0,1,100.0
2020,25,22,2,1,91.67


### 3.8 Maximum Payload Capacity Analysis

**Objective**: Identify booster versions that carried maximum payloads

In [12]:
%%sql
-- Boosters with maximum payload mass
SELECT 
    Serial,
    BoosterVersion,
    Block,
    PayloadMass,
    Orbit,
    Date,
    Outcome
FROM SPACEX_LAUNCHES
WHERE PayloadMass = (SELECT MAX(PayloadMass) FROM SPACEX_LAUNCHES)
ORDER BY Date ASC;

 * sqlite:///spacex_falcon9.db
Done.


Serial,BoosterVersion,Block,PayloadMass,Orbit,Date,Outcome
B1048,Falcon 9,5,15600.0,VLEO,2019-11-11 00:00:00,True ASDS
B1049,Falcon 9,5,15600.0,VLEO,2020-01-07 00:00:00,True ASDS
B1051,Falcon 9,5,15600.0,VLEO,2020-01-29 00:00:00,True ASDS
B1056,Falcon 9,5,15600.0,VLEO,2020-02-17 00:00:00,False ASDS
B1048,Falcon 9,5,15600.0,VLEO,2020-03-18 00:00:00,False ASDS
B1051,Falcon 9,5,15600.0,VLEO,2020-04-22 00:00:00,True ASDS
B1049,Falcon 9,5,15600.0,VLEO,2020-06-04 00:00:00,True ASDS
B1059,Falcon 9,5,15600.0,VLEO,2020-06-13 00:00:00,True ASDS
B1049,Falcon 9,5,15600.0,VLEO,2020-08-18 00:00:00,True ASDS
B1060,Falcon 9,5,15600.0,VLEO,2020-09-03 00:00:00,True ASDS


### 3.9 Block Version Evolution

**Objective**: Analyze success rates across different Falcon 9 Block versions

In [13]:
%%sql
-- Performance metrics by Block version
SELECT 
    Block,
    COUNT(*) AS Launches,
    SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) AS Successes,
    ROUND(AVG(PayloadMass), 2) AS AvgPayload,
    SUM(CASE WHEN Reused = 1 THEN 1 ELSE 0 END) AS ReusedBoosters,
    ROUND(SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) * 100.0 / 
          NULLIF(SUM(CASE WHEN Outcome NOT LIKE 'None%' THEN 1 ELSE 0 END), 0), 2) AS SuccessRate
FROM SPACEX_LAUNCHES
GROUP BY Block
ORDER BY Block ASC;

 * sqlite:///spacex_falcon9.db
Done.


Block,Launches,Successes,AvgPayload,ReusedBoosters,SuccessRate
1,19,4,2688.64,0,40.0
2,6,6,3848.17,1,100.0
3,15,11,5459.94,5,100.0
4,11,6,5089.72,4,100.0
5,117,110,9949.08,98,96.49


### Reusability Impact on Success

**Objective**: Correlate booster reusability with landing success

In [14]:
%%sql
-- Compare new vs reused booster performance
SELECT 
    CASE WHEN Reused = 1 THEN 'Reused Booster' ELSE 'New Booster' END AS BoosterStatus,
    COUNT(*) AS TotalLaunches,
    SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) AS SuccessfulLandings,
    ROUND(AVG(PayloadMass), 2) AS AvgPayload,
    ROUND(SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) * 100.0 / 
          NULLIF(SUM(CASE WHEN Outcome NOT LIKE 'None%' THEN 1 ELSE 0 END), 0), 2) AS SuccessRate
FROM SPACEX_LAUNCHES
GROUP BY BoosterStatus;

 * sqlite:///spacex_falcon9.db
Done.


BoosterStatus,TotalLaunches,SuccessfulLandings,AvgPayload,SuccessRate
New Booster,60,38,4729.66,84.44
Reused Booster,108,99,10114.09,97.06


### 3.11 Landing Technology Adoption

**Objective**: Track the evolution of landing hardware (GridFins, Legs)

In [15]:
%%sql
-- Landing technology usage over time
SELECT 
    STRFTIME('%Y', Date) AS Year,
    COUNT(*) AS TotalLaunches,
    SUM(CASE WHEN GridFins = 1 THEN 1 ELSE 0 END) AS WithGridFins,
    SUM(CASE WHEN Legs = 1 THEN 1 ELSE 0 END) AS WithLegs,
    SUM(CASE WHEN GridFins = 1 AND Legs = 1 THEN 1 ELSE 0 END) AS FullLandingHardware,
    ROUND(SUM(CASE WHEN GridFins = 1 AND Legs = 1 THEN 1 ELSE 0 END) * 100.0 / COUNT(*), 2) AS AdoptionRate
FROM SPACEX_LAUNCHES
GROUP BY Year
ORDER BY Year ASC;

 * sqlite:///spacex_falcon9.db
Done.


Year,TotalLaunches,WithGridFins,WithLegs,FullLandingHardware,AdoptionRate
2010,1,0,0,0,0.0
2012,1,0,0,0,0.0
2013,3,0,0,0,0.0
2014,6,0,2,0,0.0
2015,6,5,5,5,83.33
2016,8,8,8,8,100.0
2017,18,15,14,14,77.78
2018,18,15,15,15,83.33
2019,10,9,9,9,90.0
2020,25,24,24,24,96.0


### 3.12 Most Frequently Reused Boosters

**Objective**: Identify champion boosters with highest reuse counts

In [16]:
%%sql
-- Top reused boosters
SELECT 
    Serial,
    Block,
    MAX(ReusedCount) AS MaxReuses,
    COUNT(*) AS TotalFlights,
    SUM(CASE WHEN Outcome LIKE 'True%' THEN 1 ELSE 0 END) AS SuccessfulLandings,
    MIN(Date) AS FirstFlight,
    MAX(Date) AS LastFlight
FROM SPACEX_LAUNCHES
WHERE Serial IS NOT NULL
GROUP BY Serial
HAVING MaxReuses > 0
ORDER BY MaxReuses DESC, TotalFlights DESC
LIMIT 10;

 * sqlite:///spacex_falcon9.db
Done.


Serial,Block,MaxReuses,TotalFlights,SuccessfulLandings,FirstFlight,LastFlight
B1058,5,13,12,12,2020-05-30 00:00:00,2022-07-07 00:00:00
B1060,5,12,13,13,2020-06-30 00:00:00,2022-06-01 00:00:00
B1051,5,12,12,12,2019-03-02 00:00:00,2022-07-17 00:00:00
B1049,5,9,10,10,2018-09-10 00:00:00,2021-09-14 00:00:00
B1061,5,9,10,10,2020-11-16 00:00:00,2022-08-12 00:00:00
B1062,5,8,9,9,2020-11-05 00:00:00,2022-08-19 00:00:00
B1063,5,6,7,7,2020-11-21 00:00:00,2022-08-31 00:00:00
B1052,5,6,4,4,2022-01-31 00:00:00,2022-08-04 00:00:00
B1059,5,5,6,5,2019-12-05 00:00:00,2021-02-16 00:00:00
B1067,5,5,6,6,2021-06-03 00:00:00,2022-09-17 00:00:00


## 4. Key Findings Summary

### Launch Infrastructure
- CCSFS SLC 40 is the most utilized launch site
- Geographic distribution shows concentration in Florida facilities
- Launch site selection correlates with target orbit

### Payload Characteristics
- GTO missions carry the heaviest average payloads
- LEO/ISS missions have more consistent payload masses
- Maximum payload capacity: 15,600 kg (Starlink missions)

### Landing Success Evolution
- Clear improvement in landing success rate over time
- First successful landing: December 2015
- ASDS (drone ship) landings more common than RTLS
- Success rate exceeds 90% in later Block versions

### Booster Reusability
- Block 5 boosters demonstrate highest reuse rates
- Reused boosters maintain high success rates
- GridFins and landing legs adoption reached 100% by 2017
- Some boosters achieved 10+ successful flights

### Technology Progression
- Block 5 represents significant maturation
- Landing hardware standardization improved reliability
- Rapid iteration from Block 1 to Block 5 (2015-2018)

## 5. Implications for Predictive Modeling

### Key Predictive Features Identified

1. **Block Version**: Strong indicator of success (later blocks = higher success)
2. **Payload Mass**: Inverse relationship with RTLS success
3. **Launch Site**: Site-specific success patterns observed
4. **Orbit Type**: GTO missions more challenging for landing
5. **Reuse Status**: Minimal impact on success rate
6. **Landing Hardware**: GridFins + Legs = necessary for success
7. **Temporal Trends**: Experience improves outcomes

### Data Preparation Recommendations

1. Create binary target variable: Landing Success (True/False)
2. Engineer temporal features: Days since first launch
3. Encode categorical variables: LaunchSite, Orbit, Block
4. Handle class imbalance: More successes than failures in recent data
5. Consider train/test split by date to preserve temporal ordering

### Next Steps

1. Perform statistical correlation analysis
2. Create visualizations of key relationships
3. Build baseline classification models
4. Evaluate feature importance
5. Develop cost estimation framework

In [17]:
# Close database connection
con.close()
print("✓ Database connection closed")
print("\n" + "="*80)
print("SQL ANALYSIS COMPLETE")
print("="*80)

✓ Database connection closed

SQL ANALYSIS COMPLETE


---

## References

- **SQLite Documentation**: https://www.sqlite.org/docs.html
- **SQL Tutorial**: https://www.w3schools.com/sql/
- **Pandas SQL Integration**: https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_sql.html
- **ipython-sql**: https://github.com/catherinedevlin/ipython-sql

---

*SQL Analysis for SpaceX Falcon 9 First Stage Landing Prediction*  
*Analysis Completed: November 2025*  